<br/>

## <div style="padding:10px;background-color:#9d5a47;margin:10;color:white;font-family:newtimeroman;font-size:100%;text-align:center;border-radius: 1px 10px;overflow:hidden;font-weight:50;width:auto">Sony Churn Project - Data Preparation</div>

In [1]:
from IPython.display import Markdown, display

import pandas as pd
import numpy as np
np.random.seed(42)
rng = np.random.default_rng(42)

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'png'

from pathlib import Path
data_path = Path('./data')


In [2]:
df = pd.read_csv(data_path / "Data_Science_Challenge.csv")
df.head()

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,...,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


<br><br>

## Clean Data for modeling

In [3]:
# Change datatype of binary features to bool
for col in df.columns[df.columns.str.contains('plan')]:
    df[col] = df[col].map({'yes': True, 'no': False})
    

In [4]:
# check for string types
objs = df.dtypes[df.dtypes == 'object'].index.to_list()
objs

['state', 'phone number']

There is no guarantee that `state` will be encapsulated by `area code` and `phone number`

However, there will be some collinearity.  We choose to remove it

In [5]:
df = df.drop(['state', 'phone number'], axis=1)

### Drop `charge` columns

in our eda, we noticed that `charge` was colinear with phone usage 

In [6]:
df = df.drop(df.columns[df.columns.str.contains('charge')], axis=1)

In [7]:
df.head(3)

,account length,area code,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total eve minutes,total eve calls,total night minutes,total night calls,total intl minutes,total intl calls,customer service calls,churn
0,128,415,False,True,25,265.1,110,197.4,99,244.7,91,10.0,3,1,False
1,107,415,False,True,26,161.6,123,195.5,103,254.4,103,13.7,3,1,False
2,137,415,False,False,0,243.4,114,121.2,110,162.6,104,12.2,5,0,False


## Feature Engineering

Note there are some model specific changes we will have to make.  For instance, vmail messages will be skewed left when using a linear model like logistic regression - we would remove those without voicemail plan or we would change those without voicemail messages to the mean of those with.

In [8]:
# # add new column for df 'number vmail messages' with 'voice mail plan'
# df['num vmail messages with plan'] = df.apply(lambda x: x['number vmail messages'] if x['voice mail plan'] else 0, axis=1)
# df['num vmail messages without plan'] = df.apply(lambda x: x['number vmail messages'] if not x['voice mail plan'] else 0, axis=1)

# display((df['num vmail messages with plan'] == df['number vmail messages']).sum() == len(df), df)

#### create some new features "minutes / call"

Note that we zero out NaNs that come about due to a div/0 error.

Also note that this is a linear combination of two features.  It will be collinear with the other features.  We can probably just remove one of the other two features.

In [9]:
df['minutes per day call'] = df['total day minutes'] / df['total day calls']
df['minutes per day call'] = df['minutes per day call'].fillna(0)

df['minutes per eve call'] = df['total eve minutes'] / df['total eve calls']
df['minutes per eve call'] = df['minutes per eve call'].fillna(0)

df['minutes per night call'] = df['total night minutes'] / df['total night calls']
df['minutes per night call'] = df['minutes per night call'].fillna(0)

df['minutes per intl call'] = df['total intl minutes'] / df['total intl calls']
df['minutes per intl call'] = df['minutes per intl call'].fillna(0)

df.isna().sum().sum()

0

In [10]:
# drop `total calls` features since we have `minutes per call` features
df.drop(['total day calls', 'total eve calls', 'total night calls', 'total intl calls'], axis=1, inplace=True)

## Save to a new dataset


In [11]:
# write to a parquet file
df.to_parquet(data_path / 'Data_Science_Challenge - prepped.parquet', index=False)